In [2]:
print('hello')

hello


### How to Build a Simple Chatbot with Stored Memory using LangChain

This script demonstrates how to build a simple LLM-based chatbot with memory using LangChain. The chatbot:
- Can hold a conversation.
- Remembers previous interactions by storing memory.
- Saves memory into a JSON file for persistence.

A chatbot app is a type of software designed to simulate conversations with human users. It uses AI to interpret user input and provide relevant responses. These bots can be applied in customer service, information delivery, or task assistance. In essence, it’s like texting a smart assistant that can answer questions and help you out.

Concepts Covered:
- Chat Model vs LLM Model:
    - Chat models operate on structured messages.
    - LLM models operate on raw unstructured text.
    - Enables the chat model to retain and utilize previous interactions for context.


In [ ]:
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


from dotenv import load_dotenv, find_dotenv

In [19]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [5]:
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [9]:
from langchain_groq import ChatGroq
chatbot = ChatGroq(
    model="llama-3.3-70b-versatile"
)

In [8]:
from langchain_core.messages import HumanMessage

messages_to_the_bot = [
    HumanMessage(content="My favorite color is green")
]

In [10]:
chatbot.invoke(messages_to_the_bot)

AIMessage(content="Green is a wonderful color, often associated with nature, growth, and harmony. It's a calming and balancing color that can evoke feelings of serenity and freshness. What do you like most about the color green? Is it the way it looks in nature, or do you have a favorite green object or item?", response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 40, 'total_tokens': 104, 'completion_time': 0.232727273, 'prompt_time': 0.007710244, 'queue_time': 0.101011091, 'total_time': 0.240437517}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c4760ee73b', 'finish_reason': 'stop', 'logprobs': None}, id='run-ccb557bb-b4ae-4e12-bf34-861857e066a6-0', usage_metadata={'input_tokens': 40, 'output_tokens': 64, 'total_tokens': 104})

##### in final let's call LangSmith perform Traking

### Check if the Chatbot Remembers something

In [11]:
query = [
    HumanMessage(content='What is my favorite color ?')
    ]
chatbot.invoke(query)

AIMessage(content="I don't have any information about your favorite color. I'm a large language model, I don't have the ability to know personal preferences or details about individuals unless they're explicitly shared with me. If you'd like to share your favorite color, I'd be happy to chat with you about it!", response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 41, 'total_tokens': 103, 'completion_time': 0.225454545, 'prompt_time': 0.003359917, 'queue_time': 0.100626885, 'total_time': 0.228814462}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_90c1d253ff', 'finish_reason': 'stop', 'logprobs': None}, id='run-01728ba7-4670-4cfa-a783-68d86137cb3a-0', usage_metadata={'input_tokens': 41, 'output_tokens': 62, 'total_tokens': 103})

### Because we gonna use the Memory Buffer
    - we have tu use the lagecy API

In [12]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [13]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True,
)

In [14]:
prompt = ChatPromptTemplate(
    input_variable = ["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [15]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

C:\Users\Dan\AppData\Local\Temp\ipykernel_18628\3747834771.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(


In [16]:
chain.invoke("Hello!")

{'content': 'Hello!',
 'messages': [],
 'text': "Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?"}

In [17]:
chain.invoke("My name is Dan")

{'content': 'My name is Dan',
 'messages': [HumanMessage(content='Hello!'),
  AIMessage(content="Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?")],
 'text': "Nice to meet you, Dan. It's great to have you here. Is there something on your mind that you'd like to talk about, or would you like some conversation ideas to get us started?"}

In [18]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='Hello!'),
  AIMessage(content="Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?"),
  HumanMessage(content='My name is Dan'),
  AIMessage(content="Nice to meet you, Dan. It's great to have you here. Is there something on your mind that you'd like to talk about, or would you like some conversation ideas to get us started?")],
 'text': 'Your name is Dan. You told me that a little earlier.'}